In [1]:
#dependencies
import re
from bs4 import BeautifulSoup 
from splinter import Browser
import requests
import pandas as pd
from selenium import webdriver 
from selenium.common.exceptions import TimeoutException
from webdriver_manager.chrome import ChromeDriverManager
import json
import time
from tqdm.notebook import tqdm 

In [2]:
geo_path = "data/state_data/geo/ppp_geo.csv" #data as of 11/21/21
ppp_geo= pd.read_csv(geo_path)
ppp_geo.head()

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Unnamed: 0.1.1.1,Unnamed: 0.1.1.1.1,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,...,TRIIndustrySector,IndustrySubsector,4DigitNAICS,SBAOfficeLabel,Loan_Range,job_Range,full_add,Lat,Long,FIPS_z
0,0,0,0,0,0,9547507704,5/1/20,464,PPP,"SUMTER COATINGS, INC.",...,325 Chemicals,"3255 Paint, Coating, and Adhesive Manufacturing",3255.0,SOUTH CAROLINA DISTRICT OFFICE,"b $350,000-1 million",50 to 99,"2410 Highway 15 South, Sumter, SC",33.853682,-80.369867,NaN
1,1,1,1,1,1,9777677704,5/1/20,464,PPP,"PLEASANT PLACES, INC.",...,999 Other,5617 Services to Buildings and Dwellings,5617.0,SOUTH CAROLINA DISTRICT OFFICE,"b $350,000-1 million",50 to 99,"7684 Southrail Road, North Charleston, SC",32.942332,-80.066165,NaN
2,2,2,2,2,2,5791407702,5/1/20,1013,PPP,BOYER CHILDREN'S CLINIC,...,NaN,NaN,NaN,SEATTLE DISTRICT OFFICE,"b $350,000-1 million",50 to 99,"1850 BOYER AVE E, SEATTLE, WA",47.636067,-122.302559,NaN
3,3,3,3,3,3,6223567700,5/1/20,920,PPP,KIRTLEY CONSTRUCTION INC,...,999 Other,2361 Residential Building Construction,2361.0,SANTA ANA DISTRICT OFFICE,"b $350,000-1 million",20 to 49,"1661 MARTIN RANCH RD, SAN BERNARDINO, CA",34.224719,-117.372550,NaN
4,4,4,4,4,4,9662437702,5/1/20,101,PPP,AERO BOX LLC,...,999 Other,4842 Specialized Freight Trucking,4842.0,MASSACHUSETTS DISTRICT OFFICE,"b $350,000-1 million",20 to 49,"20101 Cornillie Dr, Roseville, MI",42.521828,-82.913680,NaN


In [3]:
ppp_geonull=ppp_geo.loc[ppp_geo["Lat"].isnull()] #457467
ppp_geonull.head()

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Unnamed: 0.1.1.1,Unnamed: 0.1.1.1.1,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,...,TRIIndustrySector,IndustrySubsector,4DigitNAICS,SBAOfficeLabel,Loan_Range,job_Range,full_add,Lat,Long,FIPS_z
369,369,369,369,369,369,5048318601,3/20/21,1084,PPS,DELTA INDUSTRIAL SERVICES INC.,...,999 Other,5511 Management of Companies and Enterprises,5511.0,ALASKA DISTRICT OFFICE,"b $350,000-1 million",10 to 19,"1229 RICHARDSON HWY, DELTA JUNCTION, AK",NaN,NaN,NaN
428,428,428,428,428,428,5085327008,4/5/20,1084,PPP,"DELTA INDUSTRIAL SERVICES, INC.",...,999 Other,5511 Management of Companies and Enterprises,5511.0,ALASKA DISTRICT OFFICE,"b $350,000-1 million",50 to 99,"1229 Richardson Hwy, DELTA JUNCTION, AK",NaN,NaN,NaN
1865,1865,1865,1865,1865,1865,4637628408,2/6/21,1084,PPS,MCKINLEY FIRE PROTECTION LLC,...,999 Other,2382 Building Equipment Contractors,2382.0,ALASKA DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"1706 Ship Ave, Anchorage, AK",NaN,NaN,NaN
2274,2274,2274,2274,2274,2274,5337318401,2/8/21,1084,PPS,THE ALASKA CENTRE FOR CREATIVE RENEWAL LLC,...,999 Other,7211 Traveler Accommodation,7211.0,ALASKA DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"46877 STILLPOINT TRL, HALIBUT COVE, AK",NaN,NaN,NaN
2379,2379,2379,2379,2379,2379,9891427104,4/15/20,1084,PPP,MCKINLEY FIRE PROTECTION,...,999 Other,2382 Building Equipment Contractors,2382.0,ALASKA DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"1706 SHIP AVENUE, ANCHORAGE, AK",NaN,NaN,NaN


In [4]:
ppp_geonull["BorrowerState"].value_counts()

TX    76183
NY    74112
FL    60040
MI    29806
WA    23754
CO    19741
MN    19730
WI    17354
IN    16601
MO    16350
LA    13668
OR    13456
UT     9295
IA     8295
NE     5804
MS     5502
HI     4980
NM     4485
ID     4416
RI     3792
ND     3222
MT     3144
PR     2952
OK     2839
SD     2695
SC     2335
WY     2139
NV     1829
KS     1645
GU      443
VI      290
MP       82
NC       11
AL        8
PA        8
AK        5
AZ        4
MA        3
NJ        3
TN        3
ME        2
IL        2
AR        2
GA        1
CT        1
OH        1
WV        1
Name: BorrowerState, dtype: int64

In [5]:
ppp_togeoRI = ppp_geonull.loc[ppp_geonull["BorrowerState"] == "RI"]
ppp_togeoRI #3792

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Unnamed: 0.1.1.1,Unnamed: 0.1.1.1.1,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,...,TRIIndustrySector,IndustrySubsector,4DigitNAICS,SBAOfficeLabel,Loan_Range,job_Range,full_add,Lat,Long,FIPS_z
782559,782559,780681,780681,779367,779367,9179517702,5/1/20,165,PPP,"COASTAL MEDICAL, INC.",...,999 Other,6211 Offices of Physicians,6211.0,RHODE ISLAND DISTRICT OFFICE,e $5-10 million,500 or more,"10 Davol Square Suite 300, Providnce, RI",NaN,NaN,NaN
782560,782560,780682,780682,779368,779368,9713307002,4/9/20,165,PPP,BROWN MEDICINE,...,999 Other,6211 Offices of Physicians,6211.0,RHODE ISLAND DISTRICT OFFICE,e $5-10 million,250 to 499,"110 ELM ST, PROVIDENCE, RI",NaN,NaN,NaN
782561,782561,780683,780683,779369,779369,5711067207,4/27/20,165,PPP,TACO INC,...,333 Machinery,"3334 Ventilation, Heating, Air-Conditioning, a...",3334.0,RHODE ISLAND DISTRICT OFFICE,e $5-10 million,250 to 499,"1160 CRANSTON ST, CRANSTON, RI",NaN,NaN,NaN
782562,782562,780684,780684,779370,779370,4819187009,4/4/20,165,PPP,"UNIVERSITY ORTHOPEDICS, INC.",...,999 Other,6211 Offices of Physicians,6211.0,RHODE ISLAND DISTRICT OFFICE,e $5-10 million,250 to 499,"1 Kettle Point Avenue, EAST PROVIDENCE, RI",NaN,NaN,NaN
782563,782563,780685,780685,779371,779371,3895377102,4/12/20,165,PPP,"H. CARR & SONS, INC.",...,999 Other,2383 Building Finishing Contractors,2383.0,RHODE ISLAND DISTRICT OFFICE,e $5-10 million,250 to 499,"100 Royal Little Drive, PROVIDENCE, RI",NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
786346,786346,784468,784468,783154,783154,8379078305,1/29/21,165,PPS,GRAPHIC INNOVATIONS INC,...,323 Printing & Publishing,3231 Printing and Related Support Activities,3231.0,RHODE ISLAND DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"380 Jefferson Blvd Ste C, Warwick, RI",NaN,NaN,NaN
786347,786347,784469,784469,783155,783155,8836008607,3/25/21,165,PPS,NEW ENGLAND DRYWALL CO.,...,999 Other,2383 Building Finishing Contractors,2383.0,RHODE ISLAND DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"816 Middle Rd Unit 5, East Greenwich, RI",NaN,NaN,NaN
786348,786348,784470,784470,783156,783156,8960777006,4/9/20,165,PPP,WATER DAMAGE DRYING SERVICES INC.,...,999 Other,2389 Other Specialty Trade Contractors,2389.0,RHODE ISLAND DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"12 GAVITT AVE, WESTERLY, RI",NaN,NaN,NaN
786349,786349,784471,784471,783157,783157,9102547103,4/15/20,165,PPP,STORMTITE COMPANY INC.,...,999 Other,2361 Residential Building Construction,2361.0,RHODE ISLAND DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"1065 WARWICK AVE, WARWICK, RI",NaN,NaN,NaN


In [6]:
#process to identify and code for unique addresses 
cols = ["LoanNumber","full_add"]
ppp_togeoRI_add = ppp_togeoRI[cols]
ppp_togeoRI_add

,LoanNumber,full_add
782559,9179517702,"10 Davol Square Suite 300, Providnce, RI"
782560,9713307002,"110 ELM ST, PROVIDENCE, RI"
782561,5711067207,"1160 CRANSTON ST, CRANSTON, RI"
782562,4819187009,"1 Kettle Point Avenue, EAST PROVIDENCE, RI"
782563,3895377102,"100 Royal Little Drive, PROVIDENCE, RI"
...,...,...
786346,8379078305,"380 Jefferson Blvd Ste C, Warwick, RI"
786347,8836008607,"816 Middle Rd Unit 5, East Greenwich, RI"
786348,8960777006,"12 GAVITT AVE, WESTERLY, RI"
786349,9102547103,"1065 WARWICK AVE, WARWICK, RI"


In [7]:
add = ppp_togeoRI["full_add"].astype(str).to_list()

In [8]:
def sleep():
    time.sleep(1)
    
executable_path = {'executable_path': ChromeDriverManager().install()}
driver = Browser('chrome', **executable_path, headless=False)    
url = 'https://geocoding.geo.census.gov/geocoder/geographies/onelineaddress?form'
driver.visit(url)

not_found = []
lat = []
long = []
geo_id = []

i=0
for i in tqdm(range(len(add))):
        driver.find_by_id('_address_id').clear()
        driver.find_by_id('_address_id').fill(add[i])
        driver.find_by_id('proceed').click()
        
        soup = BeautifulSoup(driver.html, 'html.parser')
        results = soup.find_all('div', id="pl_gov_census_geo_geocoder_domain_AddressResult")
        
        for result in results:
            result_text = result.text
            result_lat = re.findall('[^-](\d{2}[.]+\d+)', result_text)
            result_long = re.findall('[-]\d{2}[.]+\d+', result_text) #WEST 3-digit Longitude
            result_digits = re.findall('\d+', result_text)
            startswith = [s for s in result_digits if s.startswith('44')] #RI
            fips_elements =[element for element in startswith if len(element)>=15]
            
            try:
                #print(f"{add[i]} is {result_lat}, {result_long}, {fips_elements}")
                lat.append(result_lat[0])
                long.append(result_long[0])
                geo_id.append(fips_elements)
            except (KeyError, IndexError):
                not_found.append(i)
                lat.append(None)  
                long.append(None)
                geo_id.append(None)  
                print(f"{add[i]} is Not Found")
            except (TimeoutException):
                driver.navigate.refresh()
    
        sleep()
        i=i+1



====== WebDriver manager ======
Current google-chrome version is 99.0.4844
Get LATEST chromedriver version for 99.0.4844 google-chrome
Driver [/Users/ciaraspencer/.wdm/drivers/chromedriver/mac64/99.0.4844.51/chromedriver] found in cache


44 Wilclar Street, WARWICK, RI is Not Found
19 BROADWAY, NEWPORT, RI is Not Found
438 EAST MAIN ROAD IRONGATE II, MIDDLETOWN, RI is Not Found
2 Quality Drive, LINCOLN, RI is Not Found
2208 PLAINFIELD PIKE, JOHNSTON, RI is Not Found
10 RHODES AVE, NORTH SMITHFIELD, RI is Not Found
111 ENERGY WAY, WEST WARWICK, RI is Not Found
350 ESTEN AVENUE, PAWTUCKET, RI is Not Found
800 Scenic View Dr, Cumberland, RI is Not Found
LLOYD AVE, PROVIDENCE, RI is Not Found
2208 PLAINFIELD PIKE, JOHNSTON, RI is Not Found
275 WEST NATICK ROAD, SUITE 500, WARWICK, RI is Not Found
One Citizens Plaza, 8th Floor, Providence, RI is Not Found
111 Energy Way, West Warwick, RI is Not Found
10 Rhodes Ave, North Smithfield, RI is Not Found
1275 PARK EAST DR, WOONSOCKET, RI is Not Found
100 Tidal Dr, North Kingstown, RI is Not Found
100 Tidal Drive, NORTH KINGSTOWN, RI is Not Found
610 Smithfield Road, PROVIDENCE, RI is Not Found
1 Dawn Hl, Bristol, RI is Not Found
30 Sayles Hill Rd, Manville, RI is Not Found
715 Put

400 Bald Hill Rd Ste 100, Warwick, RI is Not Found
1700 W Main Rd, Middletown, RI is Not Found
1700 West Man Rd., MIDDLETOWN, RI is Not Found
3461 South County Tr Suite 202, East Greenwich, RI is Not Found
401 WALCOTT ST, PAWTUCKET, RI is Not Found
44 Cross Park Ave, North Kingstown, RI is Not Found
44 Cross Park Ave, North Kingstown, RI is Not Found
33 Appian Way, Smithfield, RI is Not Found
18 Dowling Village Blvd, North Smithfield, RI is Not Found
5 St Elizabeth Way, EAST GREENWICH, RI is Not Found
290 Smith St, North Kingstown, RI is Not Found
320 Smith Street, NORTH KINGSTOWN, RI is Not Found
191 SOCIAL ST STE 100, WOONSOCKET, RI is Not Found
33 APPIAN WAY, SMITHFIELD, RI is Not Found
275, RUMFORD, RI is Not Found
1350 Division Rd Ste 301, West Warwick, RI is Not Found
1 Austin Avenue, Greenville, RI is Not Found
15 Daniels Way, cranston, RI is Not Found
10 Business Park Dr, Smithfield, RI is Not Found
400 BALD HILL RD, WARWICK, RI is Not Found
370 Wampanoag Trail, East Providence

1672 south county trail 303, east greenwich, RI is Not Found
567 South COUNTY TRL Ste116, EXETER, RI is Not Found
One Cathedral Square, PROVIDENCE, RI is Not Found
3 Fairfield Way, WAKEFIELD, RI is Not Found
660 ORCHARD AVE, NORTH PROVIDENCE, RI is Not Found
1250 East Main Rd, PORTSMOUTH, RI is Not Found
65 SOCKANOSSET CROSSROAD, CRANSTON, RI is Not Found
1425, CRANSTON, RI is Not Found
172 - 174 WAYLAND AVE, PROVIDENCE, RI is Not Found
62 SAWYER DRIVE, COVENTRY, RI is Not Found
215 TOLL GATE RD Suite 206, WARWICK, RI is Not Found
215 Toll Gate Rd Ste 206, Warwick, RI is Not Found
P.O. Box 8492, Cranston, RI is Not Found
236 CONNELL HIGHWAT, NEWPORT, RI is Not Found
90 SCHOOL ST, FORESTDALE, RI is Not Found
6 Payne Road P.O. Box 919, BLOCK ISLAND, RI is Not Found
1 Garnett LN, GREENVILLE, RI is Not Found
7 South of Commons Road, Little Compton, RI is Not Found
400 Bald Hill Road, Suite 503, WARWICK, RI is Not Found
1525 Old Louisquisset Pike Suite 105C, Lincoln, RI is Not Found
P.O. Bo

In [9]:
print(len(add))
print(len(lat))
print(len(long))
print(len(geo_id))

3792
3792
3792
3792


In [ ]:
#pd.set_option('display.max_rows', None)
lat_df = pd.DataFrame({"Lat":lat})
lat_df

<b>Add remaining values (2801) to result_df(5461)

In [10]:
result_df = pd.DataFrame({"full_add":add, "Lat":lat, "Long":long, "FIPS_z": geo_id})
result_df 

,full_add,Lat,Long,FIPS_z
0,"10 Davol Square Suite 300, Providnce, RI",41.804245,-71.42024,[440070003022004]
1,"110 ELM ST, PROVIDENCE, RI",41.817,-71.41062,[440070006001018]
2,"1160 CRANSTON ST, CRANSTON, RI",41.795063,-71.44975,[440070141003052]
3,"1 Kettle Point Avenue, EAST PROVIDENCE, RI",41.7967,-71.37551,"[440070105021015, 440070105021015]"
4,"100 Royal Little Drive, PROVIDENCE, RI",41.84707,-71.413536,[440070029004015]
...,...,...,...,...
3787,"380 Jefferson Blvd Ste C, Warwick, RI",41.738247,-71.43955,[440030211004009]
3788,"816 Middle Rd Unit 5, East Greenwich, RI",41.645916,-71.48923,"[440030209032001, 440030209032001]"
3789,"12 GAVITT AVE, WESTERLY, RI",41.38328,-71.82086,[440090508012001]
3790,"1065 WARWICK AVE, WARWICK, RI",41.746986,-71.40541,[440030212002020]


In [11]:
result_df['FIPS_z'] = result_df['FIPS_z'].str[0]
result_df 

,full_add,Lat,Long,FIPS_z
0,"10 Davol Square Suite 300, Providnce, RI",41.804245,-71.42024,440070003022004
1,"110 ELM ST, PROVIDENCE, RI",41.817,-71.41062,440070006001018
2,"1160 CRANSTON ST, CRANSTON, RI",41.795063,-71.44975,440070141003052
3,"1 Kettle Point Avenue, EAST PROVIDENCE, RI",41.7967,-71.37551,440070105021015
4,"100 Royal Little Drive, PROVIDENCE, RI",41.84707,-71.413536,440070029004015
...,...,...,...,...
3787,"380 Jefferson Blvd Ste C, Warwick, RI",41.738247,-71.43955,440030211004009
3788,"816 Middle Rd Unit 5, East Greenwich, RI",41.645916,-71.48923,440030209032001
3789,"12 GAVITT AVE, WESTERLY, RI",41.38328,-71.82086,440090508012001
3790,"1065 WARWICK AVE, WARWICK, RI",41.746986,-71.40541,440030212002020


In [12]:
ppp_togeoRI_merge = ppp_togeoRI.merge(result_df, on="full_add", how ="left")
ppp_togeoRI_merge["Lat"] = ppp_togeoRI_merge["Lat_y"]
ppp_togeoRI_merge["Long"] = ppp_togeoRI_merge["Long_y"]
ppp_togeoRI_merge["FIPS_z"] = ppp_togeoRI_merge["FIPS_z_y"]
ppp_togeoRI_merge = ppp_togeoRI_merge.drop(["Lat_x", "Long_x", "Lat_y", "Long_y", "FIPS_z_x", "FIPS_z_y"], axis=1)
ppp_togeoRI_merge


,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Unnamed: 0.1.1.1,Unnamed: 0.1.1.1.1,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,...,TRIIndustrySector,IndustrySubsector,4DigitNAICS,SBAOfficeLabel,Loan_Range,job_Range,full_add,Lat,Long,FIPS_z
0,782559,780681,780681,779367,779367,9179517702,5/1/20,165,PPP,"COASTAL MEDICAL, INC.",...,999 Other,6211 Offices of Physicians,6211.0,RHODE ISLAND DISTRICT OFFICE,e $5-10 million,500 or more,"10 Davol Square Suite 300, Providnce, RI",41.804245,-71.42024,440070003022004
1,782560,780682,780682,779368,779368,9713307002,4/9/20,165,PPP,BROWN MEDICINE,...,999 Other,6211 Offices of Physicians,6211.0,RHODE ISLAND DISTRICT OFFICE,e $5-10 million,250 to 499,"110 ELM ST, PROVIDENCE, RI",41.817,-71.41062,440070006001018
2,782561,780683,780683,779369,779369,5711067207,4/27/20,165,PPP,TACO INC,...,333 Machinery,"3334 Ventilation, Heating, Air-Conditioning, a...",3334.0,RHODE ISLAND DISTRICT OFFICE,e $5-10 million,250 to 499,"1160 CRANSTON ST, CRANSTON, RI",41.795063,-71.44975,440070141003052
3,782562,780684,780684,779370,779370,4819187009,4/4/20,165,PPP,"UNIVERSITY ORTHOPEDICS, INC.",...,999 Other,6211 Offices of Physicians,6211.0,RHODE ISLAND DISTRICT OFFICE,e $5-10 million,250 to 499,"1 Kettle Point Avenue, EAST PROVIDENCE, RI",41.7967,-71.37551,440070105021015
4,782563,780685,780685,779371,779371,3895377102,4/12/20,165,PPP,"H. CARR & SONS, INC.",...,999 Other,2383 Building Finishing Contractors,2383.0,RHODE ISLAND DISTRICT OFFICE,e $5-10 million,250 to 499,"100 Royal Little Drive, PROVIDENCE, RI",41.84707,-71.413536,440070029004015
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5315,786346,784468,784468,783154,783154,8379078305,1/29/21,165,PPS,GRAPHIC INNOVATIONS INC,...,323 Printing & Publishing,3231 Printing and Related Support Activities,3231.0,RHODE ISLAND DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"380 Jefferson Blvd Ste C, Warwick, RI",41.738247,-71.43955,440030211004009
5316,786347,784469,784469,783155,783155,8836008607,3/25/21,165,PPS,NEW ENGLAND DRYWALL CO.,...,999 Other,2383 Building Finishing Contractors,2383.0,RHODE ISLAND DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"816 Middle Rd Unit 5, East Greenwich, RI",41.645916,-71.48923,440030209032001
5317,786348,784470,784470,783156,783156,8960777006,4/9/20,165,PPP,WATER DAMAGE DRYING SERVICES INC.,...,999 Other,2389 Other Specialty Trade Contractors,2389.0,RHODE ISLAND DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"12 GAVITT AVE, WESTERLY, RI",41.38328,-71.82086,440090508012001
5318,786349,784471,784471,783157,783157,9102547103,4/15/20,165,PPP,STORMTITE COMPANY INC.,...,999 Other,2361 Residential Building Construction,2361.0,RHODE ISLAND DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"1065 WARWICK AVE, WARWICK, RI",41.746986,-71.40541,440030212002020


In [13]:
cols = ["LoanNumber", "FIPS_z", "Lat", "Long"]
ppp_togeoRI_parsed = ppp_togeoRI_merge[cols]
ppp_togeoRI_parsed.head()

,LoanNumber,FIPS_z,Lat,Long
0,9179517702,440070003022004,41.804245,-71.42024
1,9713307002,440070006001018,41.817,-71.41062
2,5711067207,440070141003052,41.795063,-71.44975
3,4819187009,440070105021015,41.7967,-71.37551
4,3895377102,440070029004015,41.84707,-71.413536


In [14]:
ppp_geo = ppp_geo.merge(ppp_togeoRI_parsed, on="LoanNumber", how="left")
ppp_geo.head()

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Unnamed: 0.1.1.1,Unnamed: 0.1.1.1.1,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,...,SBAOfficeLabel,Loan_Range,job_Range,full_add,Lat_x,Long_x,FIPS_z_x,FIPS_z_y,Lat_y,Long_y
0,0,0,0,0,0,9547507704,5/1/20,464,PPP,"SUMTER COATINGS, INC.",...,SOUTH CAROLINA DISTRICT OFFICE,"b $350,000-1 million",50 to 99,"2410 Highway 15 South, Sumter, SC",33.853682,-80.369867,NaN,NaN,NaN,NaN
1,1,1,1,1,1,9777677704,5/1/20,464,PPP,"PLEASANT PLACES, INC.",...,SOUTH CAROLINA DISTRICT OFFICE,"b $350,000-1 million",50 to 99,"7684 Southrail Road, North Charleston, SC",32.942332,-80.066165,NaN,NaN,NaN,NaN
2,2,2,2,2,2,5791407702,5/1/20,1013,PPP,BOYER CHILDREN'S CLINIC,...,SEATTLE DISTRICT OFFICE,"b $350,000-1 million",50 to 99,"1850 BOYER AVE E, SEATTLE, WA",47.636067,-122.302559,NaN,NaN,NaN,NaN
3,3,3,3,3,3,6223567700,5/1/20,920,PPP,KIRTLEY CONSTRUCTION INC,...,SANTA ANA DISTRICT OFFICE,"b $350,000-1 million",20 to 49,"1661 MARTIN RANCH RD, SAN BERNARDINO, CA",34.224719,-117.372550,NaN,NaN,NaN,NaN
4,4,4,4,4,4,9662437702,5/1/20,101,PPP,AERO BOX LLC,...,MASSACHUSETTS DISTRICT OFFICE,"b $350,000-1 million",20 to 49,"20101 Cornillie Dr, Roseville, MI",42.521828,-82.913680,NaN,NaN,NaN,NaN


In [15]:
ppp_geo["Lat"] = ppp_geo["Lat_y"].fillna(ppp_geo["Lat_x"])
ppp_geo["Long"] = ppp_geo["Long_y"].fillna(ppp_geo["Long_x"])
ppp_geo["FIPS_z"] = ppp_geo["FIPS_z_y"].fillna(ppp_geo["FIPS_z_x"])
ppp_geo = ppp_geo.drop(["Lat_x", "Long_x", "Lat_y", "Long_y", "FIPS_z_y", "FIPS_z_x"], axis=1)
ppp_geo.head()

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Unnamed: 0.1.1.1,Unnamed: 0.1.1.1.1,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,...,TRIIndustrySector,IndustrySubsector,4DigitNAICS,SBAOfficeLabel,Loan_Range,job_Range,full_add,Lat,Long,FIPS_z
0,0,0,0,0,0,9547507704,5/1/20,464,PPP,"SUMTER COATINGS, INC.",...,325 Chemicals,"3255 Paint, Coating, and Adhesive Manufacturing",3255.0,SOUTH CAROLINA DISTRICT OFFICE,"b $350,000-1 million",50 to 99,"2410 Highway 15 South, Sumter, SC",33.8537,-80.3699,NaN
1,1,1,1,1,1,9777677704,5/1/20,464,PPP,"PLEASANT PLACES, INC.",...,999 Other,5617 Services to Buildings and Dwellings,5617.0,SOUTH CAROLINA DISTRICT OFFICE,"b $350,000-1 million",50 to 99,"7684 Southrail Road, North Charleston, SC",32.9423,-80.0662,NaN
2,2,2,2,2,2,5791407702,5/1/20,1013,PPP,BOYER CHILDREN'S CLINIC,...,NaN,NaN,NaN,SEATTLE DISTRICT OFFICE,"b $350,000-1 million",50 to 99,"1850 BOYER AVE E, SEATTLE, WA",47.6361,-122.303,NaN
3,3,3,3,3,3,6223567700,5/1/20,920,PPP,KIRTLEY CONSTRUCTION INC,...,999 Other,2361 Residential Building Construction,2361.0,SANTA ANA DISTRICT OFFICE,"b $350,000-1 million",20 to 49,"1661 MARTIN RANCH RD, SAN BERNARDINO, CA",34.2247,-117.373,NaN
4,4,4,4,4,4,9662437702,5/1/20,101,PPP,AERO BOX LLC,...,999 Other,4842 Specialized Freight Trucking,4842.0,MASSACHUSETTS DISTRICT OFFICE,"b $350,000-1 million",20 to 49,"20101 Cornillie Dr, Roseville, MI",42.5218,-82.9137,NaN


In [18]:
ppp_geo.to_csv("data/state_data/geo/ppp_geo.csv")